# Continuous Retrieval Augmentation Generation (RAG)  with HPE MLOPs Platform

author: Andrew Mendez, andrew.mendez@hpe.com

Version: 0.0.1

Date: 12.8.23

In this notebook, we see how we can create a RAG system that can automatically update as we add more data. 
We use MLDM to manage data and pipeline orchestration and TitanML + Chainlit for the user facing application.

`Pre-requisites: This demo requires an A100`

# What are we building
We are building a Retrieval Augmented Generation (RAG) system that can continuously improve with more documents.

RAG systems is combining vector databases with generative AI systems to reduce LLM hallucinatino with context.

<img src="./static/rag_ui.PNG" alt="Enterprise Machine Learning platform architecture" width="850">


# How will we build this? 
Using HPE's Machine Learning Operations (MLOps) platform
<img src="./static/platform_step3.png" alt="Enterprise Machine Learning platform architecture" width="850">

# Overview of MLOPs Pipeline

Our ML Pipline consists:
* Preproces our documents (we handle xml, csv, and pdf files)
* Add our preprocessed documents to a vector database
* We then deploy:
    * vector database (using ChromaDB)
    * an open source pretrained model (Mistral 7B Instruct) as a restful API (using TitanML)
    * and a user interface (using chainlit)

<img src="./static/deploy_rag_pipeline.PNG" alt="Enterprise Machine Learning platform architecture" width="850">


## Install pachctl and connect to pachyderm

In [1]:
# Connect to deployed pachyderm application
!pachctl connect pachd-peer.pachyderm.svc.cluster.local:30653
# list current projects
!pachctl list project

Context 'pachd-peer.pachyderm.svc.cluster.local:30653' set as active
ACTIVE PROJECT                    CREATED      DESCRIPTION
       pipeline-finbert           9 months ago Tyler - Legacy FinBERT PDK Demo
       pipeline-hpe-fsi-retrieval 8 months ago Tyler - PDK demo of for HPE FSI RAG/Retrieval Demo
       pdk-dogs-and-cats          4 months ago Tyler - Legacy Brain Dogs and Cats Demo
       pdk-brain-mri              4 months ago Tyler - Legacy Brain MRI PDK Demo
       starcoder                  4 months ago Tyler - A fine-tuned version of the huggingface starcoder model with titanML serving
       playground_tp              3 months ago Tanguy -  Pachyderm tutorial
       object-detection-demo      3 months ago -
       Test-TensorRT-LLM          3 months ago Tanguy - Testing model optimization with TensorRT-LLM and deployment with Triton
*      deploy-rag                 2 months ago -
       deploy-rag-finetune        2 months ago -
       test-catdog-pipe-test      8 weeks ag

In [2]:
!pachctl version

COMPONENT           VERSION             
pachctl             2.8.1               
pachd               0.0.0               


## Create project and set active context

In [3]:
# Create Pachyderm application
!pachctl create project deploy-rag
# Set pachctl's active context to the deploy-rag project
!pachctl config update context --project deploy-rag

project "deploy-rag" already exists
project deploy-rag already exists
editing the currently active context "pachd-peer.pachyderm.svc.cluster.local:30653"


## Create the data repo. 
* The data repo contains the documents we will ingest into the vector database and RAG system

In [54]:
!pachctl create repo data

In [55]:
!pachctl create repo code

upload documents (XML, CSV) to data repo

In [56]:
%%capture
# note xml contains folder of older press releases (2021-2022)
!pachctl put file data@master: -r -f data/HPE_press_releases/
# current press releases in csv format
!pachctl put file data@master: -r -f data/HPE_2023_Press_Releases.csv

Upload code to build RAG application

In [57]:
%%capture
!pachctl put file code@master: -r -f src/

## Process XML Pipeline

This will be deployed in `deploy-rag` mldm pipeline.

Here we define our first pipeline artiact, which is to preprocess the xml and csv documents.

In [58]:
%%writefile process_xml.yaml
pipeline:
    name: 'process_xml'
description: 'Extract content in xml files to a csv file'
input:
    cross:
        - pfs: 
            repo: 'data'
            branch: 'master'
            glob: '/'
        - pfs: 
            repo: 'code'
            branch: 'master'
            glob: '/'
transform:
    image: mendeza/python38_process:0.2
    cmd: 
        - '/bin/sh'
    stdin: 
    - 'python /pfs/code/src/py/process_xmls.py 
    --xml-directory /pfs/data/HPE_press_releases/ 
    --pdf-directory /pfs/data/ 
    --custom-csv-input /pfs/data/HPE_2023_Press_Releases.csv 
    --out-dir /pfs/out/hpe_press_releases.csv'
autoscaling: False
pod_patch: >-
  [{"op": "add","path": "/volumes/-","value": {"name":
  "host-shared","hostpath": {"path":
  "/nvmefs1/","type": "Directory"}}}, {"op":
  "add","path": "/containers/0/volumeMounts/-","value": {"mountPath":
  "/nvmefs1/","name": "host-shared"}}]

Overwriting process_xml.yaml


Deploy pipeline

In [59]:
!pachctl create pipeline -f process_xml.yaml

```bash
# Command to download resulting file from process_xml pipeline
!pachctl get file process_xml@master:hpe_press_releases.csv > hpe_press_releases.csv
```

## Add documents to vector database Pipeline
Here we define our second pipeline artiact, which is to add documents into our vector database. We take the results of the preprocessing step (process_xml) as input, so any new preprocessing runs will trigger this pipeline step.

In [60]:
%%writefile add_to_vector_db.yaml
pipeline:
    name: 'add_to_vector_db'
description: 'Extract content in xml files to a csv file'
input:
    cross:
        - pfs:
            repo: 'process_xml'
            branch: 'master'
            glob: '/'
        - pfs:
            repo: 'code'
            branch: 'master'
            glob: '/'
transform:
    image: mendeza/python38_process:0.2
    cmd: 
        - '/bin/sh'
    stdin:
    - 'pip install sentence_transformers==2.2.2'
    - 'python /pfs/code/src/py/seed.py --path_to_db /nvmefs1/test_user/cache/rag_db
    --csv_path /pfs/process_xml/hpe_press_releases.csv
    --emb_model_path /nvmefs1/test_user/cache/vector_model/e5-base-v2'
    - 'echo "$(openssl rand -base64 12)" > /pfs/out/random_file.txt'
    secrets:
        - name: pipeline-secret
          key: det_master
          env_var: DET_MASTER
        - name: pipeline-secret
          key: det_user
          env_var: DET_USER
        - name: pipeline-secret
          key: det_password
          env_var: DET_PASSWORD
        - name: pipeline-secret
          key: pac_token
          env_var: PAC_TOKEN
autoscaling: False
pod_patch: >-
  [{"op": "add","path": "/volumes/-","value": {"name":
  "host-shared","hostpath": {"path":
  "/nvmefs1/","type": "Directory"}}}, {"op":
  "add","path": "/containers/0/volumeMounts/-","value": {"mountPath":
  "/nvmefs1/","name": "host-shared"}}]

Overwriting add_to_vector_db.yaml


In [61]:
!pachctl create pipeline -f add_to_vector_db.yaml

## Deploy application Pipeline
Here we define our 3rd and final pipeline artiact, which is to deploy our finetuned LLM with our RAG system.
This step deploys our LLM as a scalable API server using TitanML and our user facing application using Chainlit. 
MLDM orchestrates allocating GPU resources needed for efficient inference. 

In [62]:
%%writefile deploy.yaml
pipeline:
    name: 'deploy'
description: 'Extract content in xml files to a csv file'
input:
    cross:
        - pfs:
            repo: 'add_to_vector_db'
            branch: 'master'
            glob: '/'
        - pfs:
            repo: 'code'
            branch: 'master'
            glob: '/'
transform:
    image: python:3.8
    cmd: 
        - '/bin/sh'
    stdin: 
        - 'bash /pfs/code/src/scripts/deploy_app.sh'
autoscaling: False
pod_patch: >-
  [{"op": "add","path": "/volumes/-","value": {"name":
  "host-shared","hostpath": {"path":
  "/nvmefs1/","type": "Directory"}}}, {"op":
  "add","path": "/containers/0/volumeMounts/-","value": {"mountPath":
  "/nvmefs1/","name": "host-shared"}}]

Overwriting deploy.yaml


In [63]:
!pachctl create pipeline -f deploy.yaml

## Now in the UI at http://10.182.1.51:8080/ , ask it the following question:
### View TitanML at http://10.182.1.48:8080/demos/playground , ask it the following question:
* "What is HPE's approach to AI?

You will see the application responds with the most relevant document!


Lets see how the RAG app will respond on information it doesn know:

* "Who is Alan May?"

We will see the RAG does not respond because it does not have this information.

Good News: we can add more documents and our system will automatically finetune and reploy the RAG system.


# Automatic retraining and deployment of RAG Application
Here we see the power of MLDM and MLDE. When we add a press release (in pdf format) abhot how long Antonio Neri has been at HPE:

In [ ]:
from IPython.core.display import HTML
def pdf(url):
    return HTML('<embed src="%s" type="application/pdf" width="100%%" height="600px" />' % url)

In [ ]:
pdf('pdf_data/output.pdf')

In [ ]:
!pachctl put file data@master: -f pdf_data/output.pdf

Lets see how the RAG app will respond on information with the updated document:
* "Who is Alan May?"
We see the model gets the answer correct!

## Clean up workspace

In [53]:
!pachctl delete pipeline deploy
!pachctl delete pipeline add_to_vector_db
!pachctl delete pipeline process_xml
!pachctl delete repo data
!pachctl delete repo code

Repo deleted.
Repo deleted.


Copy the below command on the management node to free up kubernetes resources for this demo:
* `kubectl delete -n pachyderm pod ui-pod && kubectl delete -n pachyderm pod titanml-pod && kubectl delete -n pachyderm svc ui-pod-svc && kubectl delete -n pachyderm svc titanml-pod-svc`
* `kubectl delete -n pachyderm pod ui-pod2 && kubectl delete -n pachyderm pod titanml-pod2 && kubectl delete -n pachyderm svc ui-pod-svc2 && kubectl delete -n pachyderm svc titanml-pod-svc2`